### Code

In [691]:
import random
import itertools

VARIAcaBLES = ['A', 'B', '1'] # Variable name always first followed by count(variables) - 1 times "1"
OPERAND = ['&', "|"]
negation = ['-', '']
BLOCK_SIZE = 2
PEOPLE_COUNT = 2
POPULATION_SIZE = 50
ASSUMED_ANSWER = [True, False, True] #To be set on each run


In [693]:
# Functions
def make_random_block(block_size):
    """Make boolean equation with block_size number of variables with random operators in between"""
    
    global VARIABLES, OPERAND
    
    def random_negate(nn_block):
        block = []
        for ch in nn_block:
            if ch is not '1':
                block.append(random.choice(negation)+ch)
            else:
                block.append(ch)
        return block

    # create non-negated blocks
    nn_var = random.sample(VARIABLES, block_size)

    # randomly add negation
    var = random_negate(nn_var)
    operants = random.choices(OPERAND, k=block_size-1)

    # Aleternate variables and equations into a single equation
    equation = [None]*(len(var) + len(operants))
    equation[::2], equation[1::2] = var, operants
    
    return equation

def truth_table(n):
    """Get Truth Table for `n` columns"""
    return itertools.product([True, False], repeat=n)

def similarity_score(row1, row2):
    score = 0
    for a, b in zip(row1, row2):
        if a == b:
            score+=1
        else:
            score+=-1
    return score

def random_equation(blk_size=BLOCK_SIZE, ppl_cnt=PEOPLE_COUNT):
    """Create random boolean equation"""
    eqn = []
    for _ in range(ppl_cnt):
        eqn.append(make_random_block(blk_size))
    return eqn

In [697]:
# Classes
class Genome(object):
    def __init__(self, eqn, variables=VARIABLES):
        self.eqn = eqn
        self.var_count = len(VARIABLES)-1
        self.variables = variables
        self.fitness_score = None
        self.truth_values = []
        
    def calculate_fitness(self):
        global PEOPLE_COUNT, ASSUMED_ANSWER
        score, count = 0, 0
        
        for row in truth_table(self.var_count):
            truth_value = {a:b for a, b in zip(self.variables, row)}
            # eval_result = evaluate(self.eqn, truth_value)
            eval_result = random.choice([True, False])
            if eval_result:
                self.truth_values.append(row)
                score += similarity_score(ASSUMED_ANSWER, row)
                count += 1
        if count is not 0:
            self.fitness_score = score / count
        else:
            self.fitness_score = -999.0
            
    def __repr__(self):
        eqn = " ; ".join(["".join(_) for _ in self.eqn])
        f_score = round(self.fitness_score, 3)
        repr_str = f'B_eqn : {eqn} \t fitness_score : {f_score}'
        return repr_str
        

In [694]:
# Generate Initial Population
population = []
for _ in range(POPULATION_SIZE):
    eqn = random_equation(BLOCK_SIZE, PEOPLE_COUNT)
    g = Genome(eqn)
    population.append(g)


In [696]:
# Calculate Fitness
for g in population:
    g.calculate_fitness()

In [700]:
# Genome example
g = Genome(random_equation())
g.calculate_fitness()
g

B_eqn : -B&A ; 1|A 	 fitness_score : 1.0

In [452]:
# Score Calculation examples
flag = True
while(flag):
    score = 0
    count = 0
    for row in truth_table(3):
        if(random.choice([True, False])):
            score+=similarity_score(ASSUMED_ANSWER, row)
            count += 1
    print("%2s %2s %-10s" %(score, count, score/count))
    if(score/count > 1):
        flag = False

 1  3 0.3333333333333333
 3  3 1.0       
 2  4 0.5       
-2  4 -0.5      
 4  6 0.6666666666666666
-3  7 -0.42857142857142855
 2  6 0.3333333333333333
 4  4 1.0       
-4  4 -1.0      
 3  3 1.0       
 2  4 0.5       
-2  4 -0.5      
-3  5 -0.6      
 0  6 0.0       
-1  1 -1.0      
-5  5 -1.0      
-4  4 -1.0      
 2  4 0.5       
 5  3 1.6666666666666667
